In [3]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver


In [2]:
pip install selenium

     |████████████████████████████████| 911kB 4.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### API approach, has limits per day

In [7]:
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='

def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        #print(f"Name: {user_info['name']}, Description: {user_info['description']}")
    

Requestiong:  poloclub cnn-explainer
https://api.github.com/repos/poloclub/cnn-explainer/contributors
user "xiaohk" has 189 contributions
https://github-user.now.sh/?username=xiaohk
user "rturko3" has 60 contributions
https://github-user.now.sh/?username=rturko3
user "oshaikh13" has 9 contributions
https://github-user.now.sh/?username=oshaikh13


### Full scraping approach

In [8]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [12]:
driver = webdriver.Chrome()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        #print()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
